In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from skimage import io, measure, exposure
#import image_preprocessing as pp
import pickle
import scipy.io
import h5py
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc

# This cell is all the functions needed for data preprocessing pipeline

In [2]:
# No need for keeping the label:
def read_3D_volume(DATADIR):
    """Reads and returns list of equialized histogram of images.

    Args:
      DATADIR: Directory of the images. This should be the absolute path.
    Returns:
      3D numpy array.
    """
    X = []
    for img in os.listdir(DATADIR):
        if img.endswith(".tif"):
            image = io.imread(os.path.join(DATADIR,img),as_grey=True) #Read the image
            scaler = MinMaxScaler(copy=True)
            scaler.fit(image)
            scaled_img = scaler.transform(image) # normalizing the image
            equalized_hist = exposure.equalize_hist(scaled_img)
            X.append(equalized_hist)
    X = np.array(X)
    return X

def N_sliced_box(image_arrays,label, n, SLICE_NUM, IMG_SIZE):
    """Retruns n number of randomly choosen box.
    
     Args:
        image_arrays: 3D np array of images.
        n: number of random boxs generated from this function.
        SLICE_NUM : number of slices in Z direction. default is 50 if not specified.
        IMG_SIZE: image size in X,Y directions. default is 50 if not specified.
    Returns:
        List object. ['Z','X','Y','im_array','labels'].
        Each im_array is a randomly choosen box with volume of SLICE_NUM*IMG_SIZE*IMG_SIZE.
    """
    #n_box = []
    z = np.random.randint(len(image_arrays[0])-SLICE_NUM+1, size= n)
    x = np.random.randint(len(image_arrays[1])-IMG_SIZE+1, size= n)
    y = np.random.randint(len(image_arrays[2])-IMG_SIZE+1, size= n)
    #df = pd.DataFrame(columns=['Z','X','Y','im_array','labels'])
    n_box = []
    for z,x,y in zip(z,x,y):
        box = image_arrays[z:z+SLICE_NUM,x:x+IMG_SIZE,y:y+IMG_SIZE]
        #df = df.append(pd.Series([z, x, y,box,label], index=['Z','X','Y','im_array','labels']), ignore_index=True)
        #print(" Created volume from z= {}, x = {}, y= {}".format(z,x,y))
        box = np.reshape(box, (SLICE_NUM,IMG_SIZE,IMG_SIZE, 1))
        n_box.append([z, x, y,box,label])
    return n_box

import csv
def prepare_3D_dataset(DATADIR, exporting_path, N , SLICE_NUM = 25, IMG_SIZE=50 ):
    CATEGORIES = os.listdir(DATADIR)
    print(" Reading images from directory {}, has two sub categories {}".format(DATADIR,CATEGORIES))
    data = []
    for category in CATEGORIES:
        print(" Reading {} images.".format(category))
        img_arrays = read_3D_volume(os.path.join(DATADIR,category))
        print(" Finish reading{} images. It has {} images.".format(category, len(img_arrays)))
        print(" Creating {} randomly choosen image volumes.".format(N))
        box = N_sliced_box(img_arrays, category, N, SLICE_NUM, IMG_SIZE)
        data.extend(box)
       
    random.shuffle(data)
    print('Finished creating volume data. Now saving it into hdf5 file format') 
    img_data = np.array([data[i][3] for i in range(len(data))])
    label = np.array([data[i][4] for i in range(len(data))])
    transfer_label = [np.string_(i) for i in label]
    location_data = [[data[i][0], data[i][1], data[i][2]]for i in range(len(data))]
    name = '{}_{}_{}_{}.h5'.format(N,SLICE_NUM,IMG_SIZE,IMG_SIZE)
    path = os.path.join(exporting_path,name)
    print(" Saving file with name {}, at path {}".format(name, exporting_path))
    with h5py.File(path,'w') as f:
        #f.get_config().track_order
        f.create_dataset('slice_location', data= location_data)
        f.create_dataset('img_data', data= img_data)
        #f.create_dataset('lables', data= label, dtype="S10")
        #f.attrs['labels'] = [np.string_(i) for i in label]
        f.create_dataset('labels', data= transfer_label)
        f.close()
    return

# Now let's see how to use this to create data

In [6]:
# I am creating a 1000 dataset from randomly choosen position. 
prepare_3D_dataset("CapstoneImages/","." , 3000, SLICE_NUM = 25, IMG_SIZE=50)

 Reading images from directory CapstoneImages/, has two sub categories ['A', 'V']
 Reading A images.
 Finish readingA images. It has 1001 images.
 Creating 3000 randomly choosen image volumes.
 Reading V images.
 Finish readingV images. It has 1001 images.
 Creating 3000 randomly choosen image volumes.
Finished creating volume data. Now saving it into hdf5 file format
 Saving file with name 3000_25_50_50.h5, at path .


## Okay, now we have created and stored the dataset at given path with the name 500_25_5O_50.h5 <br>
# Here I demonstrate how to open this data and how to use it.


In [3]:
filename = '3000_25_50_50.h5'  #here is the absolute path to the h5 file.
data = h5py.File(filename, 'r')  #read the h5 file into data.
list(data.keys())  # with this command you can see all the keys you have stored

['img_data', 'labels', 'slice_location']

In [4]:
X = np.asarray(list(data['img_data']))
y = list(data['labels'])
print(X.shape)
print(len(y))

(6000, 25, 50, 50, 1)
6000


In [5]:
y[:10]

[b'V', b'V', b'V', b'A', b'A', b'V', b'V', b'A', b'A', b'A']

### You might say now with your label as strings how do you feed them into CNN? 
#### Well, don't worry, here comes the hero: LabelEncoder !!!!!!!

In [6]:
le = LabelEncoder()
le.fit(y)
list(le.classes_)

[b'A', b'V']

In [7]:
l = le.transform(y) 
l

array([1, 1, 1, ..., 1, 0, 1])

# So far, we have focused on data preprocessing. Now let's move on to 3D_CNN.
I have seperated the whole CNN into 2 block. get_model, compile and fit.

In [8]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution3D, MaxPooling3D, ZeroPadding3D
from keras.optimizers import SGD

def get_model(summary=False):
    """ Return the Keras model of the network
    """
    input_shape = X.shape
    model = Sequential()
    # 1st layer group
    model.add(Convolution3D(64,(3, 3, 3), input_shape = (25,50,50,1), activation='relu', 
                            padding='same', name='conv1'))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), 
                           padding='valid', name='pool1'))
    # 2nd layer group
    model.add(Convolution3D(128,(3, 3, 3), activation='relu', 
                            padding='same', name='conv2',
                            strides=(1, 1, 1)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), 
                           padding='valid', name='pool2'))
    # 3rd layer group
    model.add(Convolution3D(256,(3, 3, 3), activation='relu', 
                            padding='same', name='conv3a',
                            strides=(1, 1, 1)))
    model.add(Convolution3D(256,(3, 3, 3), activation='relu', 
                            padding='same', name='conv3b',
                            strides=(1, 1, 1)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), 
                           padding='valid', name='pool3'))
    # 4th layer group
    model.add(Convolution3D(512,(3, 3, 3), activation='relu', 
                            padding='same', name='conv4a',
                            strides=(1, 1, 1)))
    model.add(Convolution3D(512,(3, 3, 3), activation='relu', 
                            padding='same', name='conv4b',
                            strides=(1, 1, 1)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), 
                           padding='valid', name='pool4'))
    # 5th layer group
    model.add(Convolution3D(512,(3, 3, 3), activation='relu', 
                            padding='same', name='conv5a',
                            strides=(1, 1, 1)))
    model.add(Convolution3D(512,(3, 3, 3), activation='relu', 
                            padding='same', name='conv5b',
                            strides=(1, 1, 1)))
    model.add(ZeroPadding3D(padding=(0, 1, 1)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), 
                           padding='valid', name='pool5'))
    model.add(Flatten())
    # FC layers group
    model.add(Dense(4096, activation='relu', name='fc6'))
    model.add(Dropout(.3))
    model.add(Dense(4096, activation='relu', name='fc7'))
    model.add(Dropout(.3))
    model.add(Dense(487, activation='softmax', name='fc8'))
    model.add(Dense(2, activation='softmax', name='out'))
    if summary:
        print(model.summary())
    return model



Using TensorFlow backend.


In [19]:
def get_model2(summary=False):
    """ Return the Keras model of the network
    """
    input_shape = X.shape
    model = Sequential()
    # 1st layer group
    model.add(Convolution3D(128,(5, 5, 5), input_shape = (25,50,50,1), activation='relu', 
                            padding='same', name='conv1'))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), 
                           padding='valid', name='pool1'))
    # 2nd layer group
    model.add(Convolution3D(256,(3, 3, 3), activation='relu', 
                            padding='same', name='conv2',
                            strides=(1, 1, 1)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), 
                           padding='valid', name='pool2'))
    # 3rd layer group
    model.add(Convolution3D(512,(3, 3, 3), activation='relu', 
                            padding='same', name='conv3a',
                            strides=(1, 1, 1)))
    model.add(Convolution3D(512,(3, 3, 3), activation='relu', 
                            padding='same', name='conv3b',
                            strides=(1, 1, 1)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), 
                           padding='valid', name='pool3'))
    model.add(Flatten())
    # FC layers group
    model.add(Dense(4096, activation='relu', name='fc6'))
    model.add(Dropout(.3))
    model.add(Dense(4096, activation='relu', name='fc7'))
    model.add(Dropout(.3))
    model.add(Dense(487, activation='softmax', name='fc8'))
    model.add(Dense(2, activation='softmax', name='out'))
    if summary:
        print(model.summary())
    return model



In [20]:
model = get_model2(summary=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 25, 50, 50, 128)   16128     
_________________________________________________________________
pool1 (MaxPooling3D)         (None, 25, 25, 25, 128)   0         
_________________________________________________________________
conv2 (Conv3D)               (None, 25, 25, 25, 256)   884992    
_________________________________________________________________
pool2 (MaxPooling3D)         (None, 12, 12, 12, 256)   0         
_________________________________________________________________
conv3a (Conv3D)              (None, 12, 12, 12, 512)   3539456   
_________________________________________________________________
conv3b (Conv3D)              (None, 12, 12, 12, 512)   7078400   
_________________________________________________________________
pool3 (MaxPooling3D)         (None, 6, 6, 6, 512)      0         
__________

In [23]:
def CNN_compile(loss="binary_crossentropy",optimizer='adam',metrics=['accuracy']):
    """
    This function compiles the given model with choosen loss function,
    optimizer and evaluation metrics.
    
    Args:
        loss: loss function for the model. Recommend "binary_crossentropy".
        Visit 'https://keras.io/losses/' for more information.
        optimizer: An optimizer is one of the two arguments required for compiling a Keras model.
        Visit 'https://keras.io/optimizers/' for more information.
        metric: A metric is a function that is used to judge the performance of your model. 
        Visit 'https://keras.io/metrics/' for more information. 
    Returns:
        compiled CNN model.
    """
    model = get_model(summary=False)
    model.compile(loss=loss,
                  optimizer=optimizer,
                  metrics=metrics)
    return model

In [12]:
def CNN_fit(array,label, batch_size, epochs, validation_split):
    
    le = LabelEncoder()
    le.fit(label)
    y= le.transform(label)
    
    model = CNN_compile(loss='sparse_categorical_crossentropy',
            metrics=['sparse_categorical_accuracy'])
    model.fit(array, y, batch_size= batch_size, epochs= epochs, validation_split= validation_split)
    return

# Now we have model, lets train the model.

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1001)
print(X_train.shape)
print(X_test.shape)
print(len(y_train))
print(len(y_test))

(4800, 25, 50, 50, 1)
(1200, 25, 50, 50, 1)
4800
1200


In [16]:
# This is trained on big model.
CNN_fit(X_train,y_train, 50, 5, validation_split= 0.2)

Train on 3840 samples, validate on 960 samples
Epoch 1/5
3840/3840 [==============================] - 125s 32ms/step - loss: 0.6932 - sparse_categorical_accuracy: 0.4966 - val_loss: 0.6931 - val_sparse_categorical_accuracy: 0.5094
Epoch 2/5
3840/3840 [==============================] - 115s 30ms/step - loss: 0.6932 - sparse_categorical_accuracy: 0.5029 - val_loss: 0.6930 - val_sparse_categorical_accuracy: 0.5094
Epoch 3/5
3840/3840 [==============================] - 115s 30ms/step - loss: 0.6933 - sparse_categorical_accuracy: 0.4888 - val_loss: 0.6930 - val_sparse_categorical_accuracy: 0.5094
Epoch 4/5
3840/3840 [==============================] - 115s 30ms/step - loss: 0.6933 - sparse_categorical_accuracy: 0.5029 - val_loss: 0.6930 - val_sparse_categorical_accuracy: 0.5094
Epoch 5/5
3840/3840 [==============================] - 115s 30ms/step - loss: 0.6932 - sparse_categorical_accuracy: 0.5029 - val_loss: 0.6931 - val_sparse_categorical_accuracy: 0.5094


In [26]:
history = CNN_fit(X_train,y_train, validation_split=0.25, epochs=50, batch_size=40)

Train on 3600 samples, validate on 1200 samples
Epoch 1/50


ResourceExhaustedError: OOM when allocating tensor with shape[4096,4096] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node training_3/Adam/Variable_18/Assign (defined at /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:402) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'training_3/Adam/Variable_18/Assign', defined at:
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/ubuntu/anaconda3/envs/t

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
y_score=model.predict(X_test)
fpr, tpr, thresholds = roc_curve(y_test,y_score)
plt.plot(fpr, tpr, lw=1, alpha=1)
plt.title('ROC curve')
plt.ylabel('True positive rate')
plt.xlabel('False positive rate')